# CNN PERROS Y GATOS 95% PRECISIÓN

In [ ]:
# Creación del modelo con un 95% de precisión para nuestro problema de perros y Gatos

# Vamos a añadir varias funciones que nos van a ayudar a realizar diferentes estudios de resultados que necesitabamos hacer.

# Para hacer funcionar el modelo y toda su estructura debemos tener instalado en nuestro entorno de anaconda la versión
# 2.1.0 de tensorflow, nosotros hemos usado la versión para la gráfica :

# conda install tensorflow-gpu==2.1.0

In [1]:
# Librerias necesarias para todo el proceso de entrenamiento

import sys
import tensorflow as tf
import numpy as np
import pickle
import PIL
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.activations import linear
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

print(tf.__version__)

2.1.0


In [2]:
# Función para la generación de gráficas, debemos indicar el número de épocas que ha usado el modelo para generarla 
# ajustada a los datos dados.

# Historial del entrenamiento, nombre con el que queremos guardar el modelo, número de épocas usadas.
def Visualiza_Graficas(historial, nombre, numEpochs):
    
    # Ploteamos la gráfica de perdida
    pyplot.figure(0)  
    pyplot.plot(historial.history['loss'],'r')  
    pyplot.plot(historial.history['val_loss'],'g')  
    pyplot.xticks(np.arange(0, numEpochs+1, numEpochs/10))  
    pyplot.rcParams['figure.figsize'] = (20, 20)  
    pyplot.xlabel("Num of Epochs")  
    pyplot.ylabel("Loss")  
    pyplot.title("Training Loss vs Validation Loss")  
    pyplot.legend(['train','validation'])
    pyplot.savefig('graficas/'+ nombre + '_plot1.png')
    
    # Ploteamos la gráfica de precisión precision
    pyplot.figure(1)  
    pyplot.plot(historial.history['accuracy'],'r')  
    pyplot.plot(historial.history['val_accuracy'],'g')  
    pyplot.xticks(np.arange(0, numEpochs+1, numEpochs/10))  
    pyplot.rcParams['figure.figsize'] = (20, 20)  
    pyplot.xlabel("Num of Epochs")  
    pyplot.ylabel("Accuracy")  
    pyplot.title("Training Accuracy vs Validation Accuracy")  
    pyplot.legend(['train','validation'])
    pyplot.savefig('graficas/'+ nombre +'_plot2.png')

In [3]:
# Modelo final usado, dos neuronas de salida con la función softmax
def Genera_Modelo_Final():
    # Capas de convolución
    BASELAYERS = 16
    model = Sequential()
    model.add(Conv2D(BASELAYERS, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
    model.add(AveragePooling2D((2, 2)))
    model.add(Conv2D(2*BASELAYERS, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(AveragePooling2D((2, 2)))
    model.add(Conv2D(4*BASELAYERS, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(AveragePooling2D((2, 2)))
    model.add(Conv2D(8*BASELAYERS, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(AveragePooling2D((2, 2)))
    model.add(Conv2D(16*BASELAYERS, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(AveragePooling2D((2, 2)))
   
    #Capas GAP y Dense
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(2, activation='softmax'))
    
    
    # Compilación
    opt= tf.keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# Limitación dinámica necesaria para las gráficas (A veces la memoria sobrepasa el límite y da error el entrenamiento)

### Código obtenido de la propia página de TensorFlow ###
### https://www.tensorflow.org/guide/gpu ###

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Deben aparecer las gráficas disponibles

1 Physical GPUs, 1 Logical GPUs


In [5]:
# Creamos el modelo desde la función creada anteriormente
model = Genera_Modelo_Final()

# Creamos el nombre con el que queremos guardar los datos
nombre = 'Modelo_Final'

# Creamos los callbacks para guardar en cada momento el mejor modelo obtenido y para parar el entrenamiento si se supera
# un número indicado de épocas (patience) sin mejorar el resultado (de perdida en nuestro caso)
Patience = 10
callbacks = [EarlyStopping(monitor='val_loss', patience=Patience),
             ModelCheckpoint(filepath='best_model'+nombre, monitor='val_loss', save_best_only=True)]

# Número de épocas máximas de entrenamiento
Myepochs = 100;


In [6]:
# Generamos los iteradores con las imagenes que obtenemos. Para poder usarlo debemos tener en la misma carpeta este
# notebook y la carpeta que contiene el dataser "new_dataset"

# Generador de datos de entrenamiento (Data Augmentation)
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# Generador de datos de validación, no Data Augmentation
val_datagen = ImageDataGenerator(rescale=1.0/255.0)


#Iteradores con generadores de datos diferentes.
# IMPORTANTE: class_mode = 'categorical', indicamos a TF que queremos separar las clases.

train_it = train_datagen.flow_from_directory('new_dataset/train/',
    class_mode='categorical', batch_size=64, target_size=(224, 224))
val_it = val_datagen.flow_from_directory('new_dataset/val/',
    class_mode='categorical', batch_size=64, target_size=(224, 224))

# Comprobamos que las clases están bien cogidas:
labels = (train_it.class_indices)
print(labels)

# Damos la vuelta a las etiquetas, valor númerico --> valor texto
labels = dict((v,k) for k,v in labels.items())
print(labels)

Found 18697 images belonging to 2 classes.
Found 6303 images belonging to 2 classes.
{'cats': 0, 'dogs': 1}
{0: 'cats', 1: 'dogs'}


In [ ]:
# Entrenamos el modelo y lo evaluamos. Es posible que se pare por el callback, por ello es necesario mirar en que época
# termina para generar una gráfica correcta con los datos.

# Visualizamos la estructura del modelo
model.summary();

# Entrenamos el modelo
history = model.fit(train_it, steps_per_epoch=len(train_it),
    validation_data=val_it, validation_steps=len(val_it), epochs=Myepochs, verbose=1, callbacks=callbacks)

# Evaluamos el modelo
_, acc = model.evaluate(val_it, steps=len(val_it), verbose=1, callbacks=callbacks)
print('> %.3f' % (acc * 100.0))

In [ ]:
# Debemos poner el número de épocas usadas en el entrenamiento
EpocasTotales = 64

# Visualizamos las gráficas
Visualiza_Graficas(history, nombre, EpocasTotales)

# Guardamos el modelo (último generado, el mejor se ha ido guardando automáticamente por los callbacks)
model.save('modelos/'+ nombre + '.h5')